In [1]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [2]:
import whoosh_utils

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
import bz2
import json


def save_list_bz2(data, filename):
    serialized_data = json.dumps(data)
    compressed_data = bz2.compress(serialized_data.encode("utf-8"))

    with open(filename, "wb") as f:
        f.write(compressed_data)


def load_list_bz2(filename):
    with open(filename, "rb") as f:
        compressed_data = f.read()
        decompressed_data = bz2.decompress(compressed_data)
        return json.loads(decompressed_data.decode("utf-8"))

In [4]:
import json

import polars as pl

train = pl.read_parquet("/kaggle/input/uspto-train-data-2500/train2500_seed0.parquet")
train.head(1)

publication_number,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,target_37,target_38,target_39,target_40,target_41,target_42,target_43,target_44,target_45,target_46,target_47,target_48,target_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-2011086809-…","""US-2006229256-…","""US-2009192226-…","""US-2023114071-…","""US-7754692-B2""","""US-9345727-B2""","""US-7829126-B2""","""US-2012171231-…","""US-2012171164-…","""US-2023140478-…","""US-2021138042-…","""US-9226914-B2""","""US-11389403-B2…","""US-2016029681-…","""US-10342244-B2…","""US-2021145782-…","""US-2007166411-…","""US-7867541-B2""","""US-10028519-B2…","""US-9386794-B2""","""US-2012171328-…","""US-2013338228-…","""US-2023102803-…","""US-10688070-B2…","""US-2014308393-…","""US-2016106799-…","""US-9370553-B2""","""US-2014303067-…","""US-9474766-B2""","""US-2015157048-…","""US-2018310605-…","""US-2023149542-…","""US-2017105445-…","""US-2011208153-…","""US-9345741-B2""","""US-2022386675-…","""US-2006229366-…","""US-9241923-B2""","""US-11606966-B2…","""US-2007202052-…","""US-2012171163-…","""US-2013230503-…","""US-2015305359-…","""US-2010179104-…","""US-2016015068-…","""US-2022233488-…","""US-2020230162-…","""US-2022168399-…","""US-9107894-B2""","""US-2023013644-…","""US-2014328970-…"


In [5]:
train_patents = train["publication_number"].to_list()
for i in range(50):
    train_patents += train[f"target_{i}"].to_list()
train_patents = list(set(train_patents))
len(train_patents)

125705

In [6]:
meta = pl.read_parquet("/kaggle/input/uspto-boolean-search-optimization/patent_metadata.parquet")
meta = meta.drop_nulls(subset=["publication_date"])
meta.head(1)

publication_number,publication_date,filing_date,family_id,cpc_codes
str,datetime[μs],datetime[μs],f64,list[str]
"""US-1-A""",1836-07-13 00:00:00,null,2.060279e6,"[""B61C11/04""]"


In [7]:
INDEX_SIZE = 1_000_000

train_meta = meta.filter(meta["publication_number"].is_in(train_patents))
other_meta = meta.filter(~meta["publication_number"].is_in(train_patents)).sample(
    INDEX_SIZE - len(train_patents),
    seed=46,
)
meta = pl.concat([train_meta, other_meta])
meta.shape

(1000000, 5)

In [8]:
meta = meta.with_columns(
    pl.col("publication_date").dt.year().alias("year"),
    pl.col("publication_date").dt.month().alias("month"),
)
meta.head(1)

publication_number,publication_date,filing_date,family_id,cpc_codes,year,month
str,datetime[μs],datetime[μs],f64,list[str],i32,u32
"""US-10000024-B2…",2018-06-19 00:00:00,2014-12-31 00:00:00,5.5301504e7,"[""B29C64/386"", ""B29C64/393"", … ""G05B2219/49008""]",2018,6


In [9]:
!rm -rf tmp

In [10]:
import gc
import multiprocessing
import os

from tqdm import tqdm

os.makedirs("tmp", exist_ok=True)

# with multiprocessing.Pool(5) as p:
#     _ = list(tqdm(p.imap(process_patent, meta.group_by(["year", "month"]))))
for (year, month), meta_df in tqdm(meta.group_by(["year", "month"])):
    documents = []
    this_patents_numbers = meta_df["publication_number"].to_list()
    patents_df = pl.read_parquet(
        f"/kaggle/input/uspto-boolean-search-optimization/patent_data/{year}_{month}.parquet"
    )
    common_patents = set(patents_df["publication_number"].to_list()) & set(this_patents_numbers)

    patents_df = patents_df.filter(pl.col("publication_number").is_in(common_patents))
    meta_df = meta_df.filter(pl.col("publication_number").is_in(common_patents))
    patents_df = patents_df.sort("publication_number")
    meta_df = meta_df.sort("publication_number")
    assert len(patents_df) == len(meta_df)

    for pub, ti, abs, cl, des, cpc in zip(
        meta_df["publication_number"],
        patents_df["title"],
        patents_df["abstract"],
        patents_df["claims"],
        patents_df["description"],
        meta_df["cpc_codes"].to_list(),
    ):
        doc = {
            "publication_number": pub,
            "title": ti,
            "abstract": abs,
            "claims": cl,
            "description": des,
            "cpc": cpc,
        }
        documents.append(doc)

    save_list_bz2(documents, f"tmp/{year}_{month}.json.bz2")

    del documents
    del patents_df
    del meta_df
    gc.collect()

2226it [1:04:59,  1.75s/it]


In [11]:
from glob import glob

documents = glob("tmp/*.json.bz2")

In [12]:
output_dir = "index_2500_1M"
os.makedirs(output_dir, exist_ok=True)
whoosh_utils.create_index(output_dir=output_dir, documents=documents)

100%|██████████| 2226/2226 [3:24:34<00:00,  5.51s/it]   


In [13]:
train_idx = whoosh_utils.load_index("index_2500_1M")
searcher = whoosh_utils.get_searcher(train_idx)
qp = whoosh_utils.get_query_parser()

results = whoosh_utils.execute_query("cpc:B29C64/135", qp, searcher)[:5]
results

['US-5089184-A',
 'US-2022168218-A1',
 'US-10639852-B2',
 'US-8142179-B2',
 'US-6139574-A']